In [1]:
import queue
import time
import os
import sys

from DataHandler.TradeLOBHourlyDataHandler import HistoricTradeLOBHourlyDataHandler
from Strategy.LeadLagArbitrageStrategy import LeadLagArbitrageStrategy
from Portfolio.LogPlotPortfolio import LogPlotPortfolio
from Execution.execution import SimulatedExecutionHandler

event_queue = queue.Queue()
data_handler = HistoricTradeLOBHourlyDataHandler(event_queue, 
                                           symbol_list=['btc_usdt','btc_usdt'],
                                           exchange_list=['binance','bybit'], 
                                           file_dir = 'data_sample/20240101/', 
                                           # file_dir = 'data_sample/',
                                           is_csv=False)
#sys.exit()

portfolio = LogPlotPortfolio(event_queue, data_handler)      # 组合
executor = SimulatedExecutionHandler(event_queue, data_handler)   # 回测模拟成交器；如果是实盘这里就是算法交易模块
strategy = LeadLagArbitrageStrategy(event_queue, data_handler, portfolio, executor)       # 策略实例。实际应用中应该有多个策略实例

cnt = 0
while True:
    # Update the trade/LOB (specific backtest code, as opposed to live trading)
    if data_handler.continue_backtest == True:
        data_handler.update_TradeLOB()
    else:
        break
    
    cnt+=1
    if cnt>2000: break

    # Handle the events
    while True:
        try:
            event = event_queue.get(False)
        except queue.Empty:
            break
        else:
            if event is not None:

                if event.type == 'MARKET':
                    # print('get market event', event)
                    strategy.on_market_event(event)
                    portfolio.update_holdings_from_market()
                    executor.on_market_event(event)

                elif event.type == 'ORDER':
                    print('get order event', event)
                    executor.on_order_event(event)

                elif event.type == 'FILL':
                    print('get fill event', event)
                    portfolio.update_positions_from_fill(event)
                    strategy.on_order_fill(event)
                    #sys.exit()


backtest on:  ['btc_usdt_binance', 'btc_usdt_bybit']
/*----- start initialize the DataHandler -----*/
/*----- DataHandler initialization ends -----*/

===== reload data from new hour =====
get order event {'type': 'ORDER', 'timestamp': 1704042004655, 'symbol': 'btc_usdt_bybit', 'order_id': 0, 'order_type': 'IOC', 'direction': 'BUY', 'quantity': 0.23473649707735345, 'price': 42600.959478000004}
get order event {'type': 'ORDER', 'timestamp': 1704042016379, 'symbol': 'btc_usdt_binance', 'order_id': 1, 'order_type': 'IOC', 'direction': 'BUY', 'quantity': 0.23478675512570404, 'price': 42591.84039}
get order event {'type': 'ORDER', 'timestamp': 1704042017180, 'symbol': 'btc_usdt_binance', 'order_id': 2, 'order_type': 'IOC', 'direction': 'BUY', 'quantity': 0.2347542392378292, 'price': 42597.7398}
get order event {'type': 'ORDER', 'timestamp': 1704042018403, 'symbol': 'btc_usdt_binance', 'order_id': 3, 'order_type': 'IOC', 'direction': 'BUY', 'quantity': 0.2347368827479777, 'price': 42600.8894

In [2]:
for i in data_handler.latest_symbol_exchange_trade_data_time.keys():
    print(i)

btc_usdt_binance
btc_usdt_bybit


In [3]:
'btc_usdt_bybit' in data_handler.latest_symbol_exchange_trade_data_time

True

In [4]:
data_handler.registered_symbol_exchange_trade_data['btc_usdt_binance']

AttributeError: 'HistoricTradeLOBHourlyDataHandler' object has no attribute 'registered_symbol_exchange_trade_data'

In [ ]:
data_handler.latest_symbol_exchange_trade_data_time

{'btc_usdt_binance': 1704042025507, 'btc_usdt_bybit': 1704042025312}